In [1]:
import os

import music21

bins_per_measure = 16

score_fp = input("Input directory of musicXML score files: ")
    
if os.path.isdir(score_fp):
    score_list = [os.path.join(score_fp, path) for path in os.listdir(score_fp)]
else:
    score_list = [score_fp]

Input directory of musicXML score files: C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_score_dt_transposed


In [2]:
dataset_fp = input("Input directory to store training vectors: ")

if not os.path.exists(dataset_fp):
    os.mkdir(dataset_fp)

Input directory to store training vectors: C:\Users\Danie\PycharmProjects\ChordGenerator\data\rs200_training_dt_transposed


In [4]:
for score_path in score_list:
    file_name = os.path.basename(score_path).split(".")[0]
    print(file_name)
    with open(os.path.join(dataset_fp, file_name + ".txt"), "w") as f_out:
        # TODO, method of stepping through these could be more intuitive, 
        # should be based on measure content rather than derived step sizes assuming non-overstuffed measures

        # Assume 4/4 time, then total length in quarterlength is (4/4) * 4 = 4 which is divided into bins_per_measure
        step_length = 4 / bins_per_measure 
        score = music21.converter.parse(score_path)

        melody, harmony = score.parts

        melody_measures = melody.getElementsByClass(music21.stream.Measure)
        harmony_measures = harmony.getElementsByClass(music21.stream.Measure)

        for i, mel_measure in enumerate(melody_measures):
            har_measure = harmony_measures[i]
            if mel_measure.timeSignature is not None:
                step_length = (mel_measure.timeSignature.numerator / mel_measure.timeSignature.denominator
                               * 4 / bins_per_measure)
            elif har_measure.timeSignature is not None:
                step_length = (har_measure.timeSignature.numerator / har_measure.timeSignature.denominator
                               * 4 / bins_per_measure)
            melody_elements = [item for item in mel_measure.notesAndRests]
            harmony_elements = [item for item in har_measure.notesAndRests]
            melody_index = 0
            harmony_index = 0

            for i in range(bins_per_measure):
                offset_timestep = i * step_length

                if (melody_elements[melody_index] is not melody_elements[-1] 
                    and melody_elements[melody_index + 1].offset <= offset_timestep):
                    melody_index += 1
                if (harmony_elements[harmony_index] is not harmony_elements[-1] 
                    and harmony_elements[harmony_index + 1].offset <= offset_timestep):
                    harmony_index += 1

                melody_item = melody_elements[melody_index]
                harmony_item = harmony_elements[harmony_index]

                harmony_vector = ["0" for i in range(12)]
                if not harmony_item.isRest:
                    for pitch_class in harmony_item.pitchClasses:
                        harmony_vector[pitch_class] = "1"

                f_out.write(str(melody_item.pitch.pitchClass) if melody_item.name != "rest" else "12")
                f_out.write(" ")
                f_out.write("".join(harmony_vector))
                f_out.write("\n")

            f_out.write("\n")


1999
all_along_the_watchtower
all_apologies
all_i_have_to_do_is_dream
anarchy_in_the_uk
a_change_is_gonna_come
a_day_in_the_life
a_hard_days_night
a_whiter_shade_of_pale
back_in_black
be-bop-a-lula
be_my_baby
billie_jean
bitter_sweet_symphony
bizarre_love_triangle
blitzkrieg_bop
blowin_in_the_wind
blueberry_hill
blue_suede_shoes
bohemian_rhapsody
born_to_be_wild
born_to_run
both_sides_now
bo_diddley
bridge_over_troubled_water
brown-eyed_girl
california_dreamin
california_girls
california_love
cathys_clown
changes
come_as_you_are
crazy
crying
dancing_in_the_street
dancing_queen
da_doo_ron_ron
dont_worry_baby
dream_on
earth_angel
eight_miles_high
eleanor_rigby
enter_sandman
everyday_people
every_breath_you_take
fake_plastic_trees
family_affair
fast_car
folsom_prison_blues
fortunate_son
for_what_its_worth
foxey_lady
free_fallin
georgia_on_my_mind
gimme_shelter
god_only_knows
god_save_the_queen
good_golly_miss_molly
good_vibrations
go_your_own_way
great_balls_of_fire
hallelujah
heartbreak_